In [21]:
from collections import deque
import random
import heapq

In [22]:
class BoardState:
    def __init__(self, board=None, parent=None, cost_to_root=0):
        self.cost_to_root = cost_to_root
        if board is None:
            self.board = None
        else:
            self.board = board
        if parent is None:
            self.parent = None
        else:
            self.parent = parent
        self.child_states = []
    
    # Manhattan distances for each tile. Column values correspond to 1,2,3,4,5,6,7,8, and 'x' for the blank tile. 
    # Row numbers correspond to the index of the tile, 0-9 ascending.
    distances = [
        [0, 1, 2, 3, 4, 3, 2, 1, 2],
        [1, 0, 1, 2, 3, 2, 3, 2, 1],
        [2, 1, 0, 1, 2, 3, 4, 3, 2],
        [1, 2, 3, 2, 3, 2, 1, 0, 1],
        [2, 1, 2, 1, 2, 1, 2, 1, 0],
        [3, 2, 1, 0, 1, 2, 3, 2, 1],
        [2, 3, 4, 3, 2, 1, 0, 1, 2],
        [3, 2, 3, 2, 1, 0, 1, 2, 1],
        [4, 3, 2, 1, 0, 1, 2, 3, 2]
    ]
    def __lt__(self, other): # Less than comparison for heapq to prevent a type error on tie breakers
        return self.cost_to_root < other.cost_to_root
    
    def get_heuristic_score(self):
        total_score = 0
        for index, element in enumerate(self.board):
            if element == "x":
                total_score += self.distances[index][8]
            else:
                total_score += self.distances[index][element - 1]
        return total_score
    
    def add_child_state(self, child_states):
        self.child_states.extend(child_states)

    def path_to_root(self):
        path = []
        current_state = self
        
        while current_state:
            path.append(current_state)
            current_state = current_state.parent
        
        return path[::-1]

    def locate_blank_tile(self):
        for index, element in enumerate(self.board):
            if element == "x":
                return index
        
        return -1

    def print(self):
        count = 0
        for element in self.board:
            print(element, end=" ")
            count += 1
            if count % 3 == 0:
                print()

In [23]:
def generate_moves(board_state, moveSet):
    blank_index = board_state.locate_blank_tile()
    possible_moves = []

    # tuples of move distances and their associated check if valid given the blank tile index
    moves = [(-3, blank_index - 3 >= 0), (3, blank_index + 3 <= 8),
             (-1, blank_index % 3 != 0), (1, blank_index % 3 != 2)]

    # generate new board for valid moves
    for move_distance, is_valid in moves:
        if is_valid:
            new_board = board_state.board[:]
            new_board[blank_index], new_board[blank_index + move_distance] = new_board[blank_index + move_distance], new_board[blank_index]  # tuple swap
            possible_moves.append(new_board)

    # process valid moves
    for new_board in possible_moves:
        board_str = str(new_board)
        if board_str not in moveSet:
            moveSet.add(board_str)
            child_state = BoardState(board=new_board, parent=board_state, cost_to_root=board_state.cost_to_root + 1)
            board_state.add_child_state([child_state])

In [24]:
def A_star(board_state, moveSet):
    visiting = []
    heapq.heappush(visiting, (board_state.get_heuristic_score(), board_state))

    while visiting:
        f_cost, current_state = heapq.heappop(visiting)
        
        if current_state.board == [1, 2, 3, 8, 'x', 4, 7, 6, 5]:
            return current_state.path_to_root()
        
        # duplicate checking occurs in generate_moves
        generate_moves(current_state, moveSet)

        for child_state in current_state.child_states:
            f_cost = child_state.cost_to_root + child_state.get_heuristic_score()
            heapq.heappush(visiting, (f_cost, child_state))
    return -1

In [25]:
def BFS(board, moveSet):
    visiting = deque()
    visiting.append(board)

    while visiting:
        current_state = visiting.popleft()
        
        if current_state.board == [1, 2, 3, 8, 'x', 4, 7, 6, 5]:
            return current_state.path_to_root()
        
        # duplicate checking occurs in generate_moves
        generate_moves(current_state, moveSet) 
        
        for child_state in current_state.child_states:
            visiting.append(child_state)
    return -1

In [26]:
def DFS(board_state, moveSet):
    dfs_stack = []
    dfs_stack.append(board_state)
    
    while dfs_stack:
        current_state = dfs_stack.pop()
        
        if current_state.board == [1, 2, 3, 8, 'x', 4, 7, 6, 5]:
            return current_state.path_to_root()
        
        # duplicate checking occurs in generate_moves    
        generate_moves(current_state, moveSet)
        
        for child_state in current_state.child_states:
            dfs_stack.append(child_state) 
    return -1

In [27]:
def initialize_board(boardstate):
    boardstate.board = list(range(1, 9))
    boardstate.board.append("x")
    random.shuffle(boardstate.board)

In [28]:
def main():
    new_board = BoardState()
    initialize_board(new_board)
    move_set = set()
    move_set.add(str(new_board.board))
    print("Initial Board:\n")
    new_board.print()
    
    while True:
        print("\nMenu:\n1. Initialize new board\n2. Perform BFS\n3. Perform DFS\n4. Perform A*\n5. Exit")
        choice = input("Enter choice:")
        
        if choice == '1':
            
            new_board = BoardState()
            initialize_board(new_board)
            move_set.clear()
            move_set.add(str(new_board.board))
            print("\nNew initial Board:\n")
            new_board.print()
            
        if choice == '2':
            
            print("\nBFS Search Result:")
            path = BFS(new_board, move_set)
            
            if path == -1:
                print("Initial state has no solution.")
            else:
                for board in path:
                    board.print()
                    print()
                    
            move_set.clear()
            new_board.child_states = []
            
        if choice == '3':
            
            print("\nDFS Search Result:")
            path = DFS(new_board, move_set)
            
            if path == -1:
                print("Initial state has no solution.")
            else:
                for board in path:
                    board.print()
                    print()
                    
            move_set.clear()
            new_board.child_states = []
        if choice == '4':
            
            print("\nA* Search Result:")
            path = A_star(new_board, move_set)
            
            if path == -1:
                print("Initial state has no solution.")
            else:
                for board in path:
                    board.print()
                    print()
                    
            move_set.clear()
            new_board.child_states = []

        if choice == '5':
            print("Exiting...")
            break

In [ ]:
if __name__ == "__main__":
    main()

Initial Board:

1 7 5 
2 8 x 
6 4 3 

Menu:
1. Initialize new board
2. Perform BFS
3. Perform DFS
4. Perform A*
5. Exit
